<h1>NYC Rent Affordability Investigation:<br>Gentrification ML Analysis</h1>

This is an investigation of rent affordability in NYC using neighborhood-level and borough-level from 2012-2022. 

I am using median rent data from the [Streeteasy Dashboard](https://streeteasy.com/blog/data-dashboard/), and median income data from the [U.S. Census American Community Survey](https://www.census.gov/programs-surveys/acs/data.html).

In the end, I'll use machine learning to group Brooklyn neighborhoods based on Gentrification and Affordability indicators.

# Importing packages and modules

In [1]:
# loading in custom utils
from utils.psql_connection_tools import get_engine
from utils.gentrification_tools import (engineer_gentrification_features, analyze_temporal_patterns, 
                                        detect_anomalies, cluster_gentrification_stages)

In [2]:
# stats, plotting, and access
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# postgres connection and analyses
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
# estimators/models
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
# preprocessing & engineering
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# model scoring
from sklearn.metrics import silhouette_score
# organization
import warnings
warnings.filterwarnings('ignore')

# Importing relevant data from Postgres Database

In [3]:
# creating a engine to initiate postgres connection
engine = get_engine('postgres',
                    os.getenv('PSQL_PWD'),
                    'localhost',
                    '5432',
                    'rent_affordability')

# importing median rent price and year-over-year changes in rent price data
query = """
        WITH yearly_median_rents AS (
        SELECT 
            area_name,
            borough,
            year::INTEGER AS year,
            ROUND(AVG(all_apts::INTEGER), 2) AS avg_median_all_apts,
            ROUND(AVG("1bdr_apts"::INTEGER), 2) AS avg_median_1bdr_apts,
            ROUND(AVG("3bdr_apts"::INTEGER), 2) AS avg_median_3bdr_apts
        FROM median_rent
        GROUP BY area_name, borough, year
    )
    SELECT 
        yrc.neighborhood_name AS neighborhood,
        yrc.borough_name AS borough,
        yrc.year::INTEGER AS year,
        yrc.yoy_change_pct_all,
        yrc.yoy_change_pct_1bdr,
        yrc.yoy_change_pct_3bdr,
        yrc.months_of_data,
        ymr.avg_median_all_apts,
        ymr.avg_median_1bdr_apts,
        ymr.avg_median_3bdr_apts
    FROM yoy_rent_change yrc
    JOIN yearly_median_rents ymr ON ymr.area_name = yrc.neighborhood_name
        AND ymr.borough = yrc.borough_name
        AND ymr.year = yrc.year::INTEGER 
    WHERE yrc.borough_name = 'Brooklyn' AND yrc.months_of_data = 12
    ORDER BY neighborhood, year
    """
rent_df = pd.read_sql(query, engine)
# removing row entries with fewer than a full year of data
rent_df = rent_df[rent_df['months_of_data'] >= 12]

# importing median HH income and year-over-year changes in HH income data
query = """
    SELECT 
        n.name AS neighborhood,
        yic.borough_name AS borough,
        yic.year::INTEGER AS year,
        yic.yoy_change_pct_all_hhs,
        yic.yoy_change_pct_singles,
        yic.yoy_change_pct_other_kids,
        yic.yoy_change_pct_married_kids,
        mi."income_all_HHs" AS median_all_HHs,
        mi.income_singles AS median_singles,
        mi.income_married_kids AS median_married_kids,
        mi.income_other_kids AS median_other_kids
    FROM yoy_income_change yic
    JOIN neighborhoods n ON n.name = yic.neighborhood_name
    JOIN district_neighborhoods dn ON dn.neighborhood_id = n.neighborhood_id 
    JOIN median_income mi ON mi.district_id = dn.district_id
        AND mi.year::INTEGER = yic.year::INTEGER 
    WHERE yic.borough_name = 'Brooklyn'
    ORDER BY neighborhood, year
    """
income_df = pd.read_sql(query, engine)
# handling negative values = NAN
income_df['median_all_hhs'][income_df['median_all_hhs'] < 0] = np.nan
income_df['median_singles'][income_df['median_singles'] < 0] = np.nan
income_df['median_other_kids'][income_df['median_other_kids'] < 0] = np.nan
income_df['median_married_kids'][income_df['median_married_kids'] < 0] = np.nan

# merging the income and rent datasets, to create one primate dataframe
rent_income_change_df = pd.merge(income_df, rent_df, how='inner', 
                                 on=['neighborhood','borough','year'])
# removing columns that are no longer needed
rent_income_change_df = rent_income_change_df.drop(['borough','months_of_data'], axis=1)
# reconciling duplicate neighborhood+year entries, due to PUMA district names (e.g. Crown Heights North/South)
rent_income_change_df = rent_income_change_df.groupby(['neighborhood', 'year']).mean().reset_index()
rent_income_change_df.describe()

,year,yoy_change_pct_all_hhs,yoy_change_pct_singles,yoy_change_pct_other_kids,yoy_change_pct_married_kids,median_all_hhs,median_singles,median_married_kids,median_other_kids,yoy_change_pct_all,yoy_change_pct_1bdr,yoy_change_pct_3bdr,avg_median_all_apts,avg_median_1bdr_apts,avg_median_3bdr_apts
count,72.000000,65.000000,65.000000,61.000000,65.000000,72.000000,72.000000,72.000000,72.000000,72.000000,59.000000,54.000000,72.000000,65.000000,63.000000
mean,2015.597222,4.649077,4.640769,6.493443,0.839846,51896.819444,32778.569444,24839.277778,313.861111,4.501389,5.003729,3.123889,1828.569167,1550.086769,2415.994444
std,2.543479,7.811773,17.568948,148.820443,7.147351,10225.139174,6049.836910,7246.169413,230.063696,6.041678,4.676689,5.362151,253.868962,204.042508,275.214932
min,2012.000000,-12.250000,-24.490000,-100.000000,-12.170000,32996.000000,17916.000000,11973.000000,0.000000,-7.850000,-8.930000,-15.790000,1379.000000,1198.000000,1853.600000
25%,2014.000000,0.700000,-8.400000,-47.680000,-3.360000,45185.250000,28514.250000,21541.000000,144.500000,0.267500,2.165000,0.342500,1657.815000,1411.500000,2261.950000
50%,2015.000000,3.240000,1.660000,-19.050000,0.610000,50856.500000,32573.000000,24202.000000,325.500000,5.080000,5.380000,3.495000,1801.165000,1518.200000,2388.080000
75%,2017.000000,8.820000,17.020000,24.150000,6.200000,57375.750000,37022.000000,30746.750000,419.750000,8.705000,7.505000,5.880000,1951.835000,1636.500000,2512.065000
max,2022.000000,23.770000,42.450000,1046.880000,22.560000,80471.000000,46682.000000,36025.000000,1050.000000,20.340000,23.640000,16.590000,2647.580000,2267.830000,3278.570000


In [43]:
income_df['neighborhood'].unique()

array(['Bath Beach', 'Bay Ridge', 'Bedford-Stuyvesant', 'Bensonhurst',
       'Brownsville', 'Bushwick', 'Canarsie', 'Dyker Heights',
       'East Flatbush', 'East New York', 'Flatbush', 'Flatlands',
       'Gerritsen Beach', 'Gravesend', 'Midwood', 'Sheepshead Bay',
       'Sunset Park', 'Windsor Terrace'], dtype=object)

In [41]:
shared[neighborhood for neighborhood in income_df['neighborhood'].unique() if neighborhood in rent_df['neighborhood'].unique()]

['Bath Beach',
 'Bay Ridge',
 'Bedford-Stuyvesant',
 'Bensonhurst',
 'Brownsville',
 'Bushwick',
 'Canarsie',
 'Dyker Heights',
 'East Flatbush',
 'East New York',
 'Flatbush',
 'Gravesend',
 'Midwood',
 'Sheepshead Bay',
 'Sunset Park',
 'Windsor Terrace']

In [11]:
rent_income_change_df['neighborhood'].unique()

array(['Bath Beach', 'Bay Ridge', 'Bedford-Stuyvesant', 'Bensonhurst',
       'Bushwick', 'Canarsie', 'East Flatbush', 'East New York',
       'Flatbush', 'Gravesend', 'Midwood', 'Sheepshead Bay',
       'Sunset Park', 'Windsor Terrace'], dtype=object)

# Machine Learning Gentrification Analysis

## Creating Features for the Gentrification Analysis

Creating a dataframe that contains data analyzing year-over-year changes in rent and income percentage values, affordability ratios, volatility, rent price acceleration, gentrification intensity, and other variables at the population-group and apartment-type level to develop neighborhood-level statistical information to supplement our machine learning analysis.

In [4]:
# Feature Engineering for Gentrification Detection
df_temp1 = engineer_gentrification_features(rent_income_change_df) 
print(df_temp1.columns)
df_temp1.head()

Index(['neighborhood', 'all_yoy_rent_change_mean', 'all_yoy_rent_change_std',
       'all_yoy_rent_change_max', 'all_yoy_rent_change_min',
       'all_years_high_increase', 'all_years_extreme_increase',
       'all_yoy_income_change_mean', 'all_yoy_income_change_std',
       'all_yoy_income_change_max', 'all_yoy_income_change_min',
       '1bdr_yoy_rent_change_mean', '1bdr_yoy_rent_change_std',
       '1bdr_yoy_rent_change_max', '1bdr_yoy_rent_change_min',
       '1bdr_years_high_increase', '1bdr_years_extreme_increase',
       'singles_yoy_income_change', 'singles_yoy_income_change_std',
       'singles_yoy_income_change_max', 'singles_yoy_income_change_min',
       '3bdr_yoy_rent_change_mean', '3bdr_yoy_rent_change_std',
       '3bdr_yoy_rent_change_max', '3bdr_yoy_rent_change_min',
       '3bdr_years_high_increase', '3bdr_years_extreme_increase',
       'other_kids_yoy_income_change_mean', 'other_kids_yoy_income_change_std',
       'other_kids_yoy_income_change_max', 'other_kids_yoy

,neighborhood,all_yoy_rent_change_mean,all_yoy_rent_change_std,all_yoy_rent_change_max,all_yoy_rent_change_min,all_years_high_increase,all_years_extreme_increase,all_yoy_income_change_mean,all_yoy_income_change_std,all_yoy_income_change_max,...,all_rent_income_ratio,singles_rent_income_ratio,other_kids_rent_income_ratio,married_kids_rent_income_ratio,all_gentrification_intensity,1bdr_gentrification_intensity,3bdr_gentrification_intensity,all_price_acceleration,1bdr_price_acceleration,3bdr_price_acceleration
0,Bath Beach,0.255714,7.374137,12.43,-6.10,1,0,3.154286,2.735811,5.78,...,0.406976,0.591484,120.762287,0.835607,0.167600,0.000000,0.0,12.174286,3.290000,7.754000
1,Bay Ridge,3.070000,2.701148,4.98,1.16,0,0,11.860000,NaN,11.86,...,0.359883,0.405312,181.547234,1.334071,0.000000,0.000000,0.0,1.910000,2.595000,NaN
2,Bedford-Stuyvesant,7.293333,2.778207,9.87,4.35,0,0,2.520000,16.277598,14.03,...,0.683122,0.736859,65.849642,2.500775,0.000000,0.000000,0.0,2.576667,1.273333,1.373333
3,Bensonhurst,3.735556,8.729910,20.34,-6.62,2,1,1.955556,3.729896,5.78,...,0.391778,0.560786,104.771786,0.845223,0.318274,0.000000,0.0,16.604444,6.272500,6.320000
4,Bushwick,15.000000,3.889087,17.75,12.25,2,1,23.770000,NaN,23.77,...,0.645845,0.738783,154.358496,2.451500,1.177481,0.555858,0.0,2.750000,10.050000,0.315000


Creating a dataframe that documents neighborhood-level frequency of abnormally high Year-over-Year increases in rent price (e.g. 8%, 12%, 15%, or higher.)

In [6]:
# Time Series Pattern Analysis
df_temp2 = analyze_temporal_patterns(rent_income_change_df)
df_temp2.head()

,neighborhood,all_consec_8pct,all_consec_12pct,all_consec_15pct,1bdr_consec_8pct,1bdr_consec_12pct,1bdr_consec_15pct,3bdr_consec_8pct,3bdr_consec_12pct,3bdr_consec_15pct
0,Bath Beach,1,1,0,0,0,0,1,0,0
1,Bay Ridge,0,0,0,0,0,0,0,0,0
2,Bedford-Stuyvesant,1,0,0,0,0,0,0,0,0
3,Bensonhurst,2,1,1,1,0,0,1,0,0
4,Bushwick,2,2,1,1,0,0,2,0,0


## Using Unsupervised Machine Learning to Cluster Brooklyn Neighborhoods

In [34]:
# Clustering to identify stages of gentrification
cluster_features = ['all_yoy_rent_change_mean', 'all_yoy_rent_change_std', 'all_years_high_increase', 
                    'all_price_acceleration', 'all_total_rent_change', 'all_rent_volatility', 
                    'all_total_income_change', 'all_income_volatility', 
                    'all_consec_8pct', 'all_consec_12pct', 'all_consec_15pct']
df_temp3, kmeans, fitted_scalar = cluster_gentrification_stages(df_temp1.merge(df_temp2, how='inner',
                                                                                on='neighborhood'),
                                                                cluster_features)
# viewing clustering results
df_temp3[['neighborhood','gentrification_stage']].sort_values('gentrification_stage')

Optimal number of clusters: 5


,neighborhood,gentrification_stage
6,East Flatbush,0
11,Sheepshead Bay,0
10,Midwood,0
8,Flatbush,0
12,Sunset Park,1
1,Bay Ridge,1
7,East New York,2
2,Bedford-Stuyvesant,2
4,Bushwick,3
5,Canarsie,4


In [ ]:
# 6. Interpreting Clusters and Labelling Gentrification Stages
def interpret_clusters(features_df):
    """
    Analyze cluster characteristics to label gentrification stages
    """
    cluster_summary = features_df.groupby('gentrification_stage').agg(
        {x: 'mean' for x in cluster_features}).round(2)
    
    cluster_summary.columns = [
        'avg_rent_change', 'avg_years_high_increase', 'avg_intensity',
        'avg_acceleration', 'avg_total_change', 'avg_consecutive', 'count'
    ]
    
    print("Cluster Characteristics:")
    print(cluster_summary)
    
    # Manual labeling based on characteristics (adjust based on your results)
    stage_labels = {
        0: "Stable/Pre-Gentrification",
        1: "Early Gentrification", 
        2: "Active Gentrification",
        3: "Advanced Gentrification",
        4: "Post-Gentrification/Saturated"
    }
    
    features_df['stage_label'] = features_df['gentrification_stage'].map(
        lambda x: stage_labels.get(x, f"Stage_{x}")
    )
    
    return features_df, cluster_summary

In [ ]:
# 7. Visualizing results of K-Means Cluster Analysis
def create_visualizations(features_df):
    """
    Create visualizations to understand the results
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Scatter plot: Rent change vs Gentrification intensity
    axes[0,0].scatter(
        features_df['rent_change_mean'], 
        features_df['gentrification_intensity'],
        c=features_df['gentrification_stage'], 
        cmap='viridis', 
        alpha=0.7
    )
    axes[0,0].set_xlabel('Average YoY Rent Change %')
    axes[0,0].set_ylabel('Gentrification Intensity')
    axes[0,0].set_title('Neighborhoods by Gentrification Stage')
    
    # Box plot of rent changes by cluster
    features_df.boxplot(
        column='rent_change_mean', 
        by='stage_label', 
        ax=axes[0,1]
    )
    axes[0,1].set_title('Rent Change Distribution by Stage')
    
    # Anomaly detection visualization
    colors = ['red' if x else 'blue' for x in features_df['is_anomaly']]
    axes[1,0].scatter(
        features_df['gentrification_intensity'],
        features_df['anomaly_score'],
        c=colors,
        alpha=0.7
    )
    axes[1,0].set_xlabel('Gentrification Intensity')
    axes[1,0].set_ylabel('Anomaly Score')
    axes[1,0].set_title('Anomaly Detection (Red = Anomalies)')
    
    # Years of high increases
    features_df.boxplot(
        column='years_high_increase',
        by='stage_label',
        ax=axes[1,1]
    )
    axes[1,1].set_title('Years of High Increases by Stage')
    
    plt.tight_layout()
    plt.show()

Using the Isolation Forest ML technique to identify neighborhoods with anomalous patterns in rent/income volatility, rent price acceleration, total changes in Household income, and more:

In [7]:
# Detecting anomalies across All Apartment and All Household Data 
anomaly_features = ['all_yoy_rent_change_mean', 'all_yoy_rent_change_std', 'all_years_high_increase', 
                             'all_price_acceleration', 'all_total_rent_change', 'all_rent_volatility', 
                             'all_total_income_change', 'all_income_volatility']
df_temp3 = detect_anomalies(df_temp1, anomaly_features, 
                            'all_apts_anomaly_score', 'all_apts_is_anomaly')
df_temp3.head()

,neighborhood,all_yoy_rent_change_mean,all_yoy_rent_change_std,all_yoy_rent_change_max,all_yoy_rent_change_min,all_years_high_increase,all_years_extreme_increase,all_yoy_income_change_mean,all_yoy_income_change_std,all_yoy_income_change_max,...,other_kids_rent_income_ratio,married_kids_rent_income_ratio,all_gentrification_intensity,1bdr_gentrification_intensity,3bdr_gentrification_intensity,all_price_acceleration,1bdr_price_acceleration,3bdr_price_acceleration,all_apts_anomaly_score,all_apts_is_anomaly
0,Bath Beach,0.255714,7.374137,12.43,-6.10,1,0,3.154286,2.735811,5.78,...,120.762287,0.835607,0.167600,0.000000,0.0,12.174286,3.290000,7.754000,0.025089,False
1,Bay Ridge,3.070000,2.701148,4.98,1.16,0,0,11.860000,NaN,11.86,...,181.547234,1.334071,0.000000,0.000000,0.0,1.910000,2.595000,NaN,0.002876,False
2,Bedford-Stuyvesant,7.293333,2.778207,9.87,4.35,0,0,2.520000,16.277598,14.03,...,65.849642,2.500775,0.000000,0.000000,0.0,2.576667,1.273333,1.373333,0.042079,False
3,Bensonhurst,3.735556,8.729910,20.34,-6.62,2,1,1.955556,3.729896,5.78,...,104.771786,0.845223,0.318274,0.000000,0.0,16.604444,6.272500,6.320000,-0.014744,True
4,Bushwick,15.000000,3.889087,17.75,12.25,2,1,23.770000,NaN,23.77,...,154.358496,2.451500,1.177481,0.555858,0.0,2.750000,10.050000,0.315000,-0.001233,True


In [ ]:
# Step 8: Main execution function
def main():
    """
    Execute the complete gentrification analysis pipeline
    """
    print("Step 1: Loading data...")
    # df = load_and_prepare_data()  # Uncomment when you have DB connection
    
    # For demonstration, create sample data
    np.random.seed(42)
    neighborhoods = ['Park Slope', 'Williamsburg', 'Crown Heights', 'Bushwick', 
                    'Red Hook', 'DUMBO', 'Bedford-Stuyvesant', 'Greenpoint']
    
    # Create sample data - replace with your actual data loading
    sample_data = []
    for neighborhood in neighborhoods:
        for year in range(2015, 2024):
            sample_data.append({
                'neighborhood_name': neighborhood,
                'borough_name': 'Brooklyn',
                'year': year,
                'yoy_rent_change_pct': np.random.normal(5 + hash(neighborhood) % 10, 3),
                'yoy_income_change_pct': np.random.normal(2, 2),
                'median_rent': 2000 + year * 100 + hash(neighborhood) % 500,
                'median_income': 50000 + year * 1000 + hash(neighborhood) % 10000
            })
    
    df = pd.DataFrame(sample_data)
    
    print("Step 2: Engineering features...")
    features_df = engineer_gentrification_features(df)
    
    print("Step 3: Analyzing temporal patterns...")
    temporal_features = analyze_temporal_patterns(df)
    features_df = features_df.merge(temporal_features, on='neighborhood_name')
    
    print("Step 4: Detecting anomalies...")
    features_df = detect_anomalies(features_df)
    
    print("Step 5: Clustering neighborhoods...")
    features_df, model, scaler = cluster_gentrification_stages(features_df)
    
    print("Step 6: Interpreting clusters...")
    features_df, cluster_summary = interpret_clusters(features_df)
    
    print("Step 7: Creating visualizations...")
    create_visualizations(features_df)
    
    print("\nResults Summary:")
    print(features_df[['neighborhood_name', 'stage_label', 'is_anomaly', 
                      'gentrification_intensity', 'anomaly_score']].sort_values('gentrification_intensity', ascending=False))
    
    return features_df, model, scaler

if __name__ == "__main__":
    results, model, scaler = main()